In [14]:
import csv
import random
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold

# Load a CSV file
def loadCsv(filename):
    trainSet = []
    with open(filename, 'r') as file:
        lines = csv.reader(file)
        dataset = list(lines)
        headers = dataset[0]  # Get the column headers
        for row in dataset[1:]:  # Skip the first row (headers)
            for j in range(4):
                row[j] = float(row[j])
            trainSet.append(row)
    return trainSet

# Rescale dataset columns to the range 0-1
def normalize(dataset, minmax):
    for row in dataset:
        for i in range(len(row) - 1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

# Convert string column to float
def column_to_float(dataset, column):
    for row in dataset:
        try:
            row[column] = float(row[column])
        except ValueError:
            print("Error with row", column, ":", row[column])
            pass

# Convert string column to integer
def column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = []
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = []
        while len(fold) < fold_size:
            index = random.randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calculate accuracy percentage
def accuracy_met(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def run_algorithm(dataset, n_folds, *args):
    kf = KFold(n_splits=n_folds, shuffle=True)
    scores = []
    for train_index, test_index in kf.split(dataset):
        train_set = [dataset[i] for i in train_index]
        test_set = [dataset[i] for i in test_index]
        predicted = back_propagation(train_set, test_set, *args)
        actual = [row[-1] for row in test_set]
        accuracy = accuracy_met(actual, predicted)
        cm = confusion_matrix(actual, predicted)
        print(cm)
        scores.append(accuracy)
    return scores




# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden, k0, k1):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs, k0, k1)
    predictions = []
    for row in test:
        prediction = predict(network, row)
        predictions.append(prediction)
    return predictions

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights': [random.uniform(-1, 1) for _ in range(n_inputs + 1)]} for _ in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights': [random.uniform(-1, 1) for _ in range(n_hidden + 1)]} for _ in range(n_outputs)]
    network.append(output_layer)
    return network

# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

# Transfer neuron activation
def transfer(activation, k0, k1):
    return k0 + k1 * activation

# Forward propagate input to a network output
def forward_propagate(network, row, k0, k1):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation, k0, k1)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

# Calculate the derivative of a neuron output
def transfer_derivative(output, k1):
    return k1 * output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected, k1):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'], k1)

# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs, k0, k1):
    for epoch in range(n_epoch):
        for row in train:
            outputs = forward_propagate(network, row, k0, k1)
            expected = [0 for _ in range(n_outputs)]
            expected[row[-1]] = 1
            backward_propagate_error(network, expected, k1)
            update_weights(network, row, l_rate)

# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row, k0, k1)
    return outputs.index(max(outputs))


In [15]:
# Test the backpropagation algorithm
seed(1)
# Load and prepare data
filename = 'iris.csv'
dataset = loadCsv(filename)
for i in range(len(dataset[0]) - 1):
    column_to_float(dataset, i)
# Convert class column to integers
lookup = column_to_int(dataset, len(dataset[0]) - 1)

def dataset_minmax(dataset):
    minmax = []
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        min_value = min(col_values)
        max_value = max(col_values)
        minmax.append((min_value, max_value))
    return minmax

# Normalize input variables
minmax = dataset_minmax(dataset)
normalize(dataset, minmax)

# Define the search space for hyperparameters
search_space = {
    'l_rate': Real(0.01, 0.3, prior='log-uniform'),
    'n_epoch': Integer(50, 200),
    'n_hidden': Integer(1, 5),
    'k0': Real(-2, 2),
    'k1': Real(-2, 2)
}

# Define the BayesSearchCV
opt = BayesSearchCV(back_propagation, search_space, n_iter=50, n_jobs=-1, cv=5)

# Separate the dataset into input features and labels
X = np.array(dataset)[:, :-1]
y = np.array(dataset)[:, -1]

# Perform the hyperparameter search
opt.fit(X, y)

# Get the best hyperparameters
best_hyperparams = opt.best_params_
l_rate = best_hyperparams['l_rate']
n_epoch = best_hyperparams['n_epoch']
n_hidden = best_hyperparams['n_hidden']
k0 = best_hyperparams['k0']
k1 = best_hyperparams['k1']

# Evaluate the algorithm using cross validation
n_folds = 5
scores = run_algorithm(dataset, n_folds, l_rate, n_epoch, n_hidden, k0, k1)
print('Scores:', scores)
print('Mean Accuracy: %.3f%%' % (sum(scores) / float(len(scores))))

TypeError: estimator should be an estimator implementing 'fit' method, <function back_propagation at 0x000001DB703885E0> was passed